In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load Data

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_PATH = '/content/drive/MyDrive/Projects/Llama2-Medical-Chatbot/71763-gale-encyclopedia-of-medicine.-vol.-1.-2nd-ed.pdf'
DB_FAISS_PATH = '/content/drive/MyDrive/Projects/Llama2-Medical-Chatbot/vectorstore'

# Create vector database
def create_vector_db():
    loader = PyPDFLoader(DATA_PATH)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

if __name__ == "__main__":
    create_vector_db()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import textwrap
import re

DB_FAISS_PATH = '/content/drive/MyDrive/Projects/Llama2-Medical-Chatbot/vectorstore'
QUERY = "Your question here"

def query_vector_db(query):
    # Load the FAISS vector store with dangerous deserialization allowed
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)

    # Search the vector store
    results = db.similarity_search(query, k=5)  # k is the number of top results to return
    return results

def clean_text(text):
    # Split the text into sentences
    sentences = text.split('. ')

    # Remove leading and trailing whitespace from each sentence
    sentences = [sentence.strip() for sentence in sentences]

    # Join the sentences back together, ensuring proper punctuation
    cleaned_text = '. '.join(sentences)

    # Add a period at the end if it's missing
    if not cleaned_text.endswith('.'):
        cleaned_text += '.'

    # Handle bullet points by ensuring they start on a new line
    cleaned_text = re.sub(r'•', r'\n•', cleaned_text)

    # Handle headings by assuming they are in uppercase and followed by a period
    cleaned_text = re.sub(r'([A-Z\s]+)\.', lambda m: '\n' + m.group(1).title() + '.', cleaned_text)

    return cleaned_text

def left_align_text(text, width):
    # Wrap text to the specified width without adding extra spaces
    wrapped_text = textwrap.fill(text, width=width)
    return wrapped_text

if __name__ == "__main__":
    query = "What is abortion?"  # Replace with your question
    results = query_vector_db(query)

    # Concatenate results
    concatenated_result = " ".join([clean_text(result.page_content) for result in results])

    # Left align the concatenated result
    left_aligned_result = left_align_text(concatenated_result, 80)

    # Display the left-aligned result
    print(left_aligned_result)


Abortion has been a legal procedure in the United Statessince 1973. Purpose An
abortion may be performed whenever there is some compelling reason to end a
pregnancy. Womenhave abortions because continuing the pregnancy wouldcause them
hardship, endanger their life or health, orbecause prenatal testing has shown
that the fetus will beborn with severe abnormalities. Abortions are safest when
performed within the first. mother in carrying and giving birth to more than one
ortwo babies, and also to decrease the risk of complicationsto the remaining
fetus(es). The term selective abortionalso refers to choosing to abort a fetus
for reasons such asthe woman is carrying a fetus which likely will be bornwith
some birth defect or impairment, or because the sexof the fetus is not preferred
by the individual. Purpose A woman may decide to abort for health reasons, for.
Multi-fetal pregnancy —A pregnancy of two or more fetuses. Selective reduction
—Typically referred to in cases of multifetal pregn

In [2]:
!pip install langchain
!pip install langchain_community
!pip install pypdf
!pip install sentence-transformers
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 